### Save model to file by Pickle
(link tiếng anh: http://scikit-learn.org/stable/modules/model_persistence.html#model-persistence)

Có thể dùng thư viện __pickle__ được dựng sẵn trong Python để lưu model trong scikit vào file

In [8]:
from sklearn import svm
from sklearn import datasets
clf = svm.SVC()

#load example dataset iris
X,y = datasets.load_iris(return_X_y=True)

#Train model
clf.fit(X,y)

#Save model
import pickle
s = pickle.dumps(clf)
clf2 = pickle.loads(s)

print (clf)
print ("-----------------------------")
print (clf2)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
-----------------------------
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


Có thể dùng thư viện joblib thay thế cho pickle(joblib.dump & joblib.load) (which is more efficient on objects that carry _large numpy arrays_ internally as is often the case for fitted scikit-learn estimators, but can only pickle to the disk and not to a string:)

In [11]:
from sklearn.externals import joblib
joblib.dump(clf, "filename.pkl")

clf3 = joblib.load("filename.pkl")
print (clf3)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


## Những giới hạn về mặt bảo mật và bảo trì
pickle (and joblib by extension) có một số vấn đề liên quan đến vấn đề bảo mật và bảo trì. Vì vậy:
- Không bao giờ unpickle dữ liệu không đáng tin cậy, nó có thể dẫn đến mã độc được thực thi khi tải. (vde bảo mật)
- Mặc dù các model được lưu bằng phiên bản scikit-learn có thể tải trong các phiên bản scikit-learn khác nhưng điều này hoàn toàn không được hỗ trợ. Vị vậy nên lưu ý rằng các hoạt động được thực hiện trên dữ liệu đó có thể mang lại kết quả khác mong đợi. (vde bảo trì)
    - Vì vậy để có thể rebuild model với future versions của sklearn, nên save additional metadata sau cùng với pickled model:
        - The training data, e.g. a reference to a immutable snapshot
        - The python source code used to generate the model (src code python đã dùng để generate model)
        - The versions of scikit-learn and its dependencies
        - The cross validation score obtained on the training data (This should make it possible to check that the cross-validation score is in the same range as before.)
        - __Tuy nhiên, do bên trong model có thể khác nhau trên hai kiến trúc khác nhau, việc dumping một mô hình trên một kiến trúc và load nó lên một kiến trúc khác không được hỗ trợ.__

If you want to know more about these issues and explore other possible serialization methods, please refer to th
is [talk by Alex Gaynor](http://pyvideo.org/pycon-us-2014/pickles-are-for-delis-not-software.html).